In [2]:
import pandas as pd

filepath = r"https://github.com/data-to-insight/ERN-sessions/raw/main/data/903_xlsx_errors.xlsx"
 
dfs = pd.read_excel(filepath, sheet_name=None)

In [3]:
print(dfs.keys())

dict_keys(['ad1', 'episodes', 'header', 'missing', 'oc2', 'oc3', 'placed_for_adoption', 'previous_permenance', 'reviews', 'uasc'])


In [4]:
dfs['ad1'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   CHILD        30 non-null     object
 1   DOB          30 non-null     object
 2   DATE_INT     30 non-null     object
 3   DATE_MATCH   30 non-null     object
 4   FOSTER_CARE  30 non-null     int64 
 5   NB_ADOPTR    30 non-null     object
 6   SEX_ADOPTR   30 non-null     object
 7   LS_ADOPTR    30 non-null     object
dtypes: int64(1), object(7)
memory usage: 2.0+ KB


In [5]:
# rule 102 = DOB is not a valid date
# .notna() / .isna()
header = dfs['header']
header['DOB'] = pd.to_datetime(header['DOB'], format="%d/%m/%Y", errors='coerce')

# this is do identify rows with errors
#invalid_dob = header['DOB'].isna()
#error_rows = header[invalid_dob]

# this identifies correct rows and then says show 'not' valid
vaild_dobs = header['DOB'].notna()
error_rows = header[~vaild_dobs]

error_rows

,CHILD,SEX,DOB,ETHNIC,UPN,MOTHER,MC_DOB
2675,bad_entry_1,7,NaT,Alien,UPDOG,NaN,NaN


In [6]:
# rule 115 - date of LA decision child placed is not a date

p4a = dfs['placed_for_adoption']

p4a['DATE_PLACED'] = pd.to_datetime(p4a['DATE_PLACED'], format="%d/%m/%Y", errors='coerce')

invalid_p4a = p4a['DATE_PLACED'].isna()
error_p4a = p4a[invalid_p4a]

error_p4a

,CHILD,DOB,DATE_PLACED,DATE_PLACED_CEASED,REASON_PLACED_CEASED
29,bad_entry_1,Not a date,NaT,This also isn't a date,oop


In [7]:
# .isin()
# returns rows where value is in a list of values
# value == another_value (only allows one value) using | can be too repetive

# rule 103 - ethnicity code is not valid or entered

codes = [
        "WBRI",
        "WIRI",
        "WOTH",
        "WIRT",
        "WROM",
        "MWBC",
        "MWBA",
        "MWAS",
        "MOTH",
        "AIND",
        "APKN",
        "ABAN",
        "AOTH",
        "BCRB",
        "BAFR",
        "BOTH",
        "CHNE",
        "OOTH",
        "REFU",
        "NOBT",
    ]

valid_eth_codes = header['ETHNIC'].isin(codes)

error_eth = header[~valid_eth_codes]

error_eth

,CHILD,SEX,DOB,ETHNIC,UPN,MOTHER,MC_DOB
2675,bad_entry_1,7,NaT,Alien,UPDOG,NaN,NaN


In [8]:
#114 - Data entry to record the status of former carer(s) of an adopted child is invalid.
# table is ad1, column is FOSTER_CARE, 0, 1

ad1 = dfs['ad1']

ad1['FOSTER_CARE'] = ad1['FOSTER_CARE'].astype('str')

#ad1.info()

f_type = ['0','1']

valid_foster = ad1['FOSTER_CARE'].isin(f_type)

error_foster = ad1[~valid_foster]

error_foster

,CHILD,DOB,DATE_INT,DATE_MATCH,FOSTER_CARE,NB_ADOPTR,SEX_ADOPTR,LS_ADOPTR
29,bad_entry,datedate,notdate,datenot,777,Q,Will,let's go


In [9]:


df = pd.merge(p4a.reset_index(), header, how='left', on='CHILD', suffixes=["_p4a", None])

invalid_date_placed = df['DATE_PLACED'] <= df['DOB']

error_df = df[invalid_date_placed]

error_df

,index,CHILD,DOB_p4a,DATE_PLACED,DATE_PLACED_CEASED,REASON_PLACED_CEASED,SEX,DOB,ETHNIC,UPN,MOTHER,MC_DOB


In [10]:

# 302 - First episode starts before child was born.
# header, episodes, DOB from header, DECOM from episodes
 
episodes= dfs['episodes']
 
episodes['DECOM'] = pd.to_datetime(episodes['DECOM'], dayfirst=True, errors='coerce')
 
df = pd.merge(episodes, header, how='left', on='CHILD', suffixes=["_epi", "_hd"])
 
invalid_decom = df['DOB'] >= df['DECOM']
 
error_rows = df[invalid_decom]
 
error_rows


,CHILD,DECOM,RNE,LS,CIN,PLACE,PLACE_PROVIDER,DEC,REC,REASON_PLACE_CHANGE,HOME_POST,PL_POST,URN,SEX,DOB,ETHNIC,UPN,MOTHER,MC_DOB


In [11]:
# 188 - Child is aged under 4 years at
# the end of the year but a Strengths and Difficulties (SDQ) score
# or a reason for no SDQ score has been completed
 
oc2 = dfs['oc2'][['CHILD', 'DOB', 'SDQ_SCORE', "SDQ_REASON"]].copy()
 
oc2['DOB'] = pd.to_datetime(oc2['DOB'], dayfirst=True, errors='coerce')
 
collection_end = pd.to_datetime("31/03/2020", dayfirst=True)
 
oc2['4bday'] = oc2['DOB'] + pd.DateOffset(years=4)
 
under_4 = oc2['4bday'] > collection_end
 
sdq_score_reason = oc2['SDQ_REASON'].notna() | oc2['SDQ_SCORE']
 
error_rows = oc2[under_4 & sdq_score_reason]
 
error_rows

,CHILD,DOB,SDQ_SCORE,SDQ_REASON,4bday
2675,bad_child,2024-12-05,oops,bad,2028-12-05


In [13]:
# 189 - Child is aged 17 years or over at the beginning of the year,
#  but an Strengths and Difficulties (SDQ) score or a reason for no
# Strengths and Difficulties (SDQ) score has been completed.
# 01/04/2019

oc2 = dfs['oc2'][['CHILD', 'DOB', 'SDQ_SCORE', "SDQ_REASON"]].copy()
 
oc2['DOB'] = pd.to_datetime(oc2['DOB'], dayfirst=True, errors='coerce')
 
collection_start = pd.to_datetime("01/04/2019", dayfirst=True)
 
oc2['17bday'] = oc2['DOB'] + pd.DateOffset(years=17)
 
over_17 = oc2['17bday'] < collection_start
 
sdq_score_reason = oc2['SDQ_REASON'].notna() | oc2['SDQ_SCORE']
 
error_rows = oc2[over_17 & sdq_score_reason]
 
error_rows

,CHILD,DOB,SDQ_SCORE,SDQ_REASON,17bday
20,947046,2001-08-22,NaN,SDQ5,2018-08-22
33,714655,2001-10-01,NaN,SDQ2,2018-10-01
35,819792,2001-04-14,25,NaN,2018-04-14
43,197474,2001-03-30,5,NaN,2018-03-30
58,552166,2001-11-09,34,NaN,2018-11-09
...,...,...,...,...,...
2634,97512,2001-12-27,NaN,SDQ1,2018-12-27
2653,413076,2001-05-14,32,NaN,2018-05-14
2659,482205,2001-04-26,32,NaN,2018-04-26
2660,473732,2001-06-17,NaN,SDQ1,2018-06-17
